# Fugue + Dask-sql

When `dask-sql` is installed, Fugue can automatically detect and set it as the default Dask execution engine. You need to pip install [fugue-jupyter](https://pypi.org/project/fugue-jupyter/) in order to use the magic `%fsql`.

If you don't restart Jupyter notebook/lab after pip install fugue-jupyter, you need to do

```python
from fugue_jupyter import setup
setup()
```

to enable the `%fsql` magic right away but without sql highlight.

In [ ]:
from dask.distributed import Client

client = Client(processes=True)

In [3]:
%%fsql dask
CREATE [["xyz"], ["xxx"]] SCHEMA a:str
SELECT * WHERE a LIKE '%y%'
PRINT

,a
0,xyz


You can yield the dataframes to use out of the cell

In [4]:
%%fsql dask
src = CREATE [["xyz"], ["xxx"]] SCHEMA a:str

a = SELECT a AS b WHERE a LIKE '%y%'
    YIELD DATAFRAME AS test

b = SELECT CONCAT(a, '-') AS b FROM src WHERE a LIKE '%xx%'
    YIELD DATAFRAME AS test1

SELECT * FROM a UNION SELECT * FROM b
PRINT

,b
0,xyz
1,xxx-


In [5]:
test.native  # a Dask DataFrame

,b
npartitions=2,
,object
,...
,...


In [6]:
test1.native.compute()

,b
1,xxx-


You can also do it in the programmatical approach and you can also directly pass in a Dask `Client` as the execution engine

In [7]:
from fugue_sql import fsql

fsql("""
CREATE [["xyz"], ["xxx"]] SCHEMA a:str
SELECT * WHERE a LIKE '%y%'
PRINT
""").run(client)

,a
0,xyz


DataFrames()

In [8]:
from fugue_sql import fsql

result = fsql("""
CREATE [["xyz"], ["xxx"]] SCHEMA a:str
SELECT * WHERE a LIKE '%y%'
YIELD DATAFRAME AS test2
""").run(client)

In [9]:
result["test2"].native  # a Dask DataFrame

,a
npartitions=2,
,object
,...
,...
